In [1]:
!ln -s /kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images ./filelists/CUB/images
# !ln -s /kaggle/input/miniimagenet/miniImageNet/images ./filelists/miniImagenet/images
# !ln -s /kaggle/input/d/arjun2000ashok/tieredimagenet/tiered_imagenet/ ./filelists/tieredImagenet/images
# !ln -s /kaggle/input/stanford-dogs-dataset/images/Images ./filelists/dogs/Images
# !ln -s /kaggle/input/stanford-cars-dataset/cars_train/cars_train ./filelists/cars/images 
# !ln -s /kaggle/input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images ./filelists/aircrafts/images
# !ln -s /kaggle/input/vggflowers/images ./filelists/flowers/images

In [1]:
!python train.py --dataset CUB --train_aug --method baseline

  File "train.py", line 133
    base_datamgr            = SetDataManager(image_size, n_query = n_query,  **train_few_shot_params, isAircraft=isAircraft)
                                                                                                    ^
SyntaxError: invalid syntax
